<a href="https://colab.research.google.com/github/mpiatek26/Fewshot_text_classification_with_meta_learning_and_BERT/blob/main/reptile_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, RandomSampler
from transformers import BertForSequenceClassification
from copy import deepcopy
import numpy as np
from sklearn.metrics import accuracy_score
import logging
import gc

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class Learner(nn.Module):
    """
    Meta Learner class for few-shot learning using BERT.
    """

    def __init__(self, args):
        super(Learner, self).__init__()
        # Initialize the model with the provided arguments
        self.init_model(args)

    def init_model(self, args):
        """
        Initialize the model and its components.
        """
        self.num_labels = args.num_labels
        self.outer_batch_size = args.outer_batch_size
        self.inner_batch_size = args.inner_batch_size
        self.outer_update_lr = args.outer_update_lr
        self.inner_update_lr = args.inner_update_lr
        self.inner_update_step = args.inner_update_step
        self.inner_update_step_eval = args.inner_update_step_eval
        self.bert_model = args.bert_model

        # Load BERT model for sequence classification
        self.model = BertForSequenceClassification.from_pretrained(self.bert_model, num_labels=self.num_labels)
        self.outer_optimizer = Adam(self.model.parameters(), lr=self.outer_update_lr)

        # Set the model to training mode by default
        self.model.train()

    def forward(self, batch_tasks, training=True):
        """
        Forward pass for batch of tasks.
        """
        task_accs = []
        sum_gradients = []
        num_task = len(batch_tasks)
        num_inner_update_step = self.inner_update_step if training else self.inner_update_step_eval

        for task_id, (support, query) in enumerate(batch_tasks):
            # Process each task
            fast_model, inner_optimizer = self.clone_model()
            self.train_inner_loop(fast_model, inner_optimizer, support, num_inner_update_step, task_id)

            # Evaluate the fast (adapted) model
            task_acc = self.evaluate_task(fast_model, query)
            task_accs.append(task_acc)

            if training:
                # Aggregate gradients from the fast model
                self.aggregate_gradients(fast_model, sum_gradients, task_id)

            # Clean up to free memory
            del fast_model, inner_optimizer
            torch.cuda.empty_cache()

        if training:
            # Update the original model
            self.update_model(sum_gradients, num_task)

        return np.mean(task_accs)

    def clone_model(self):
        """
        Clone the main model for fast adaptation.
        """
        fast_model = deepcopy(self.model).to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        inner_optimizer = Adam(fast_model.parameters(), lr=self.inner_update_lr)
        fast_model.train()
        return fast_model, inner_optimizer

    def train_inner_loop(self, fast_model, inner_optimizer, support, num_inner_update_step, task_id):
        """
        Inner training loop for fast adaptation of the model.
        """
        support_dataloader = DataLoader(support, sampler=RandomSampler(support), batch_size=self.inner_batch_size)
        for _ in range(num_inner_update_step):
            for batch in support_dataloader:
                loss = self.compute_loss(fast_model, batch)
                loss.backward()
                inner_optimizer.step()
                inner_optimizer.zero_grad()

    def compute_loss(self, fast_model, batch):
        """
        Compute loss for a batch of data.
        """
        batch = tuple(t.to(fast_model.device) for t in batch)
        input_ids, attention_mask, segment_ids, label_id = batch
        outputs = fast_model(input_ids, attention_mask, segment_ids, labels=label_id)
        return outputs[0]

    def evaluate_task(self, fast_model, query):
        """
        Evaluate the fast model on the query set.
        """
        fast_model.eval()
        query_dataloader = DataLoader(query, sampler=None, batch_size=len(query))
        query_batch = iter(query_dataloader).next()
        query_batch = tuple(t.to(fast_model.device) for t in query_batch)
        q_input_ids, q_attention_mask, q_segment_ids, q_label_id = query_batch

        with torch.no_grad():
            q_outputs = fast_model(q_input_ids, q_attention_mask, q_segment_ids, labels=q_label_id)
            q_logits = nn.functional.softmax(q_outputs[1], dim=1)
            pre_label_id = torch.argmax(q_logits, dim=1).detach().cpu().numpy()
            q_label_id = q_label_id.detach().cpu().numpy()

            acc = accuracy_score(pre_label_id, q_label_id)
            return acc

    def aggregate_gradients(self, fast_model, sum_gradients, task_id):
        """
        Aggregate gradients from the fast model for meta-update.
        """
        meta_weights = list(self.model.parameters())
        fast_weights = list(fast_model.parameters())

        for i, (meta_params, fast_params) in enumerate(zip(meta_weights, fast_weights)):
            gradient = meta_params - fast_params
            if task_id == 0:
                sum_gradients.append(gradient)
            else:
                sum_gradients[i] += gradient

    def update_model(self, sum_gradients, num_task):
        """
        Update the original model using the aggregated gradients.
        """
        for i, params in enumerate(self.model.parameters()):
            params.grad = sum_gradients[i] / float(num_task)

        self.outer_optimizer.step()
        self.outer_optimizer.zero_grad()

        del sum_gradients
        gc.collect()
